# train (Confidential)

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define constants
IMAGE_SIZE = (128, 128)
LATENT_DIM = 32
BATCH_SIZE = 50
EPOCHS = 500
RECONSTRUCTION_LOSS_MULTIPLIER = 0.5

# Function to load and preprocess images
def load_and_preprocess_images(image_folder):
    image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=IMAGE_SIZE)
        img = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return np.array(images)

# Load and preprocess images
train_images = load_and_preprocess_images("train_dsl")

# Define the encoder part of VAE
encoder_inputs = layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = encoder_inputs
for filters in [32, 64, 128, 256]:
    x = layers.Conv2D(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)  # Added dense layer for projection
z_mean = layers.Dense(LATENT_DIM, activation='relu')(x)
z_log_var = layers.Dense(LATENT_DIM, activation='relu')(x)

# Define sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.shape(z_mean)[0], LATENT_DIM), mean=0., stddev=1.)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Define the decoder part of VAE
decoder_inputs = layers.Input(shape=(LATENT_DIM,))
x = layers.Dense(512, activation='relu')(decoder_inputs)  # Dense layer for decoding
x = layers.Dense(8*8*256, activation='relu')(x)  # Projection
x = layers.Reshape((8, 8, 256))(x)
for filters in [256, 128, 64, 32]:
    x = layers.Conv2DTranspose(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.UpSampling2D(size=(2, 2))(x)
decoder_outputs = layers.Conv2DTranspose(3, 4, activation='sigmoid', padding='same')(x)

# Define the VAE model
encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
decoder = models.Model(decoder_inputs, decoder_outputs, name='decoder')

outputs = decoder(encoder(encoder_inputs)[2])
vae = models.Model(encoder_inputs, outputs, name='vae')

# Define the loss function for VAE
def vae_loss(inputs, outputs, z_mean, z_log_var):
    reconstruction_loss = tf.keras.losses.binary_crossentropy(tf.keras.backend.flatten(inputs), tf.keras.backend.flatten(outputs))
    reconstruction_loss *= IMAGE_SIZE[0] * IMAGE_SIZE[1] * 3
    kl_loss = 1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var)
    kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = tf.keras.backend.mean(reconstruction_loss + RECONSTRUCTION_LOSS_MULTIPLIER * kl_loss)
    return vae_loss

# Compile the VAE model
vae.compile(optimizer='adam', loss=lambda x, y: vae_loss(encoder_inputs, outputs, z_mean, z_log_var))

# Train the VAE model
vae.fit(train_images, train_images, epochs=EPOCHS, batch_size=BATCH_SIZE)

# Extract embeddings from the encoder part
embeddings = encoder.predict(train_images)[0]  # Assuming you want z_mean

# Save or use embeddings as needed
np.save("train.npy", embeddings)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/500
1000/1000 [==============================] - 182s 182ms/sample - loss: 830226706034573.7500
Epoch 2/500
1000/1000 [==============================] - 167s 167ms/sample - loss: 14545.9097
Epoch 3/500
1000/1000 [==============================] - 165s 165ms/sample - loss: 13250.5692
Epoch 4/500
1000/1000 [==============================] - 166s 166ms/sample - loss: 12793.4508
Epoch 5/500
1000/1000 [==============================] - 166s 166ms/sample - loss: 12553.5985
Epoch 6/500
1000/1000 [==============================] - 164s 164ms/sample - loss: 12389.7359
Epoch 7/500
1000/1000 [==============================] - 160s 160ms/sample - loss: 12105.8499
Epoch 8/500
1000/1000 [==============================] - 165s 165ms/sample - loss: 11948.8304
Epoch 9/500
1000/1000 [===

1000/1000 [==============================] - 165s 165ms/sample - loss: 10913.7095
Epoch 81/500
1000/1000 [==============================] - 164s 164ms/sample - loss: 10871.4813
Epoch 82/500
1000/1000 [==============================] - 164s 164ms/sample - loss: 10960.1139
Epoch 83/500
1000/1000 [==============================] - 165s 165ms/sample - loss: 10913.7970
Epoch 84/500
1000/1000 [==============================] - 163s 163ms/sample - loss: 10980.8746
Epoch 85/500
1000/1000 [==============================] - 166s 166ms/sample - loss: 10911.0901
Epoch 86/500
1000/1000 [==============================] - 162s 162ms/sample - loss: 10915.0262
Epoch 87/500
1000/1000 [==============================] - 163s 163ms/sample - loss: 10922.4544
Epoch 88/500
1000/1000 [==============================] - 163s 163ms/sample - loss: 10923.9109
Epoch 89/500
1000/1000 [==============================] - 164s 164ms/sample - loss: 10886.0841
Epoch 90/500
1000/1000 [==============================] - 164s 

1000/1000 [==============================] - 165s 165ms/sample - loss: 10860.1776
Epoch 166/500
1000/1000 [==============================] - 158s 158ms/sample - loss: 10822.5825
Epoch 167/500
1000/1000 [==============================] - 165s 165ms/sample - loss: 10834.4248
Epoch 168/500
1000/1000 [==============================] - 162s 162ms/sample - loss: 10764.0885
Epoch 169/500
1000/1000 [==============================] - 161s 161ms/sample - loss: 10761.7577
Epoch 170/500
1000/1000 [==============================] - 164s 164ms/sample - loss: 10763.3026
Epoch 171/500
1000/1000 [==============================] - 162s 162ms/sample - loss: 10785.2100
Epoch 172/500
1000/1000 [==============================] - 160s 160ms/sample - loss: 10795.7179
Epoch 173/500
1000/1000 [==============================] - 165s 165ms/sample - loss: 10723.2854
Epoch 174/500
1000/1000 [==============================] - 149s 149ms/sample - loss: 10779.7616
Epoch 175/500
1000/1000 [=============================

1000/1000 [==============================] - 162s 162ms/sample - loss: 10749.8003
Epoch 251/500
1000/1000 [==============================] - 163s 163ms/sample - loss: 10703.4140
Epoch 252/500
1000/1000 [==============================] - 163s 163ms/sample - loss: 10704.9817
Epoch 253/500
1000/1000 [==============================] - 165s 165ms/sample - loss: 10736.1711
Epoch 254/500
1000/1000 [==============================] - 162s 162ms/sample - loss: 10742.0923
Epoch 255/500
1000/1000 [==============================] - 162s 162ms/sample - loss: 10777.1919
Epoch 256/500
1000/1000 [==============================] - 161s 161ms/sample - loss: 10731.7513
Epoch 257/500
1000/1000 [==============================] - 164s 164ms/sample - loss: 10686.6039
Epoch 258/500
1000/1000 [==============================] - 161s 161ms/sample - loss: 10719.3815
Epoch 259/500
1000/1000 [==============================] - 161s 161ms/sample - loss: 10736.8756
Epoch 260/500
1000/1000 [=============================

1000/1000 [==============================] - 75s 75ms/sample - loss: 10721.0443
Epoch 337/500
1000/1000 [==============================] - 75s 75ms/sample - loss: 10727.9483
Epoch 338/500
1000/1000 [==============================] - 75s 75ms/sample - loss: 10723.9288
Epoch 339/500
1000/1000 [==============================] - 75s 75ms/sample - loss: 10645.0043
Epoch 340/500
1000/1000 [==============================] - 73s 73ms/sample - loss: 10757.2001
Epoch 341/500
1000/1000 [==============================] - 75s 75ms/sample - loss: 10724.1329
Epoch 342/500
1000/1000 [==============================] - 75s 75ms/sample - loss: 10765.3264
Epoch 343/500
1000/1000 [==============================] - 75s 75ms/sample - loss: 10703.2388
Epoch 344/500
1000/1000 [==============================] - 74s 74ms/sample - loss: 10714.0934
Epoch 345/500
1000/1000 [==============================] - 74s 74ms/sample - loss: 10725.0027
Epoch 346/500
1000/1000 [==============================] - 74s 74ms/sample

1000/1000 [==============================] - 46s 46ms/sample - loss: 10626.9484
Epoch 424/500
1000/1000 [==============================] - 47s 47ms/sample - loss: 10658.5288
Epoch 425/500
1000/1000 [==============================] - 47s 47ms/sample - loss: 10697.1527
Epoch 426/500
1000/1000 [==============================] - 47s 47ms/sample - loss: 10720.9043
Epoch 427/500
1000/1000 [==============================] - 47s 47ms/sample - loss: 10708.1094
Epoch 428/500
1000/1000 [==============================] - 47s 47ms/sample - loss: 10649.8048
Epoch 429/500
1000/1000 [==============================] - 47s 47ms/sample - loss: 10677.4699
Epoch 430/500
1000/1000 [==============================] - 47s 47ms/sample - loss: 10602.2642
Epoch 431/500
1000/1000 [==============================] - 47s 47ms/sample - loss: 10697.4279
Epoch 432/500
1000/1000 [==============================] - 47s 47ms/sample - loss: 10702.7949
Epoch 433/500
1000/1000 [==============================] - 47s 47ms/sample

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Load the embeddings from the .npy file
embeddings = np.load("train.npy")

# Assuming the embeddings are 3-dimensional
x = embeddings[:, 0]
y = embeddings[:, 1]
z = embeddings[:, 2]

# Plot the embeddings in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, c='b', marker='o')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()

<Figure size 640x480 with 1 Axes>

# val

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define constants
IMAGE_SIZE = (128, 128)
LATENT_DIM = 32
BATCH_SIZE = 50
EPOCHS = 500
RECONSTRUCTION_LOSS_MULTIPLIER = 0.5

# Function to load and preprocess images
def load_and_preprocess_images(image_folder):
    image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=IMAGE_SIZE)
        img = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return np.array(images)

# Load and preprocess images
train_images = load_and_preprocess_images("val_dsl")

# Define the encoder part of VAE
encoder_inputs = layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = encoder_inputs
for filters in [32, 64, 128, 256]:
    x = layers.Conv2D(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)  # Added dense layer for projection
z_mean = layers.Dense(LATENT_DIM, activation='relu')(x)
z_log_var = layers.Dense(LATENT_DIM, activation='relu')(x)

# Define sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.shape(z_mean)[0], LATENT_DIM), mean=0., stddev=1.)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Define the decoder part of VAE
decoder_inputs = layers.Input(shape=(LATENT_DIM,))
x = layers.Dense(512, activation='relu')(decoder_inputs)  # Dense layer for decoding
x = layers.Dense(8*8*256, activation='relu')(x)  # Projection
x = layers.Reshape((8, 8, 256))(x)
for filters in [256, 128, 64, 32]:
    x = layers.Conv2DTranspose(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.UpSampling2D(size=(2, 2))(x)
decoder_outputs = layers.Conv2DTranspose(3, 4, activation='sigmoid', padding='same')(x)

# Define the VAE model
encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
decoder = models.Model(decoder_inputs, decoder_outputs, name='decoder')

outputs = decoder(encoder(encoder_inputs)[2])
vae = models.Model(encoder_inputs, outputs, name='vae')

# Define the loss function for VAE
def vae_loss(inputs, outputs, z_mean, z_log_var):
    reconstruction_loss = tf.keras.losses.binary_crossentropy(tf.keras.backend.flatten(inputs), tf.keras.backend.flatten(outputs))
    reconstruction_loss *= IMAGE_SIZE[0] * IMAGE_SIZE[1] * 3
    kl_loss = 1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var)
    kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = tf.keras.backend.mean(reconstruction_loss + RECONSTRUCTION_LOSS_MULTIPLIER * kl_loss)
    return vae_loss

# Compile the VAE model
vae.compile(optimizer='adam', loss=lambda x, y: vae_loss(encoder_inputs, outputs, z_mean, z_log_var))

# Train the VAE model
vae.fit(train_images, train_images, epochs=EPOCHS, batch_size=BATCH_SIZE)

# Extract embeddings from the encoder part
embeddings = encoder.predict(train_images)[0]  # Assuming you want z_mean

# Save or use embeddings as needed
np.save("val.npy", embeddings)


Epoch 1/500
1000/1000 [==============================] - 49s 49ms/sample - loss: nan       
Epoch 2/500
1000/1000 [==============================] - 46s 46ms/sample - loss: nan
Epoch 3/500
1000/1000 [==============================] - 47s 47ms/sample - loss: nan
Epoch 4/500
1000/1000 [==============================] - 46s 46ms/sample - loss: nan
Epoch 5/500
1000/1000 [==============================] - 47s 47ms/sample - loss: nan
Epoch 6/500
1000/1000 [==============================] - 47s 47ms/sample - loss: nan
Epoch 7/500
1000/1000 [==============================] - 46s 46ms/sample - loss: nan
Epoch 8/500
1000/1000 [==============================] - 46s 46ms/sample - loss: nan
Epoch 9/500
1000/1000 [==============================] - 46s 46ms/sample - loss: nan
Epoch 10/500
1000/1000 [==============================] - 46s 46ms/sample - loss: nan
Epoch 11/500
1000/1000 [==============================] - 47s 47ms/sample - loss: nan
Epoch 12/500
1000/1000 [==============================] 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Load the embeddings from the .npy file
embeddings = np.load("val.npy")

# Assuming the embeddings are 3-dimensional
x = embeddings[:, 0]
y = embeddings[:, 1]
z = embeddings[:, 2]

# Plot the embeddings in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, c='b', marker='o')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()

# PD_test (PD)

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define constants
IMAGE_SIZE = (128, 128)
LATENT_DIM = 32
BATCH_SIZE = 50
EPOCHS = 500
RECONSTRUCTION_LOSS_MULTIPLIER = 0.5

# Function to load and preprocess images
def load_and_preprocess_images(image_folder):
    image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=IMAGE_SIZE)
        img = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return np.array(images)

# Load and preprocess images
train_images = load_and_preprocess_images("PD_test_dsl")

# Define the encoder part of VAE
encoder_inputs = layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = encoder_inputs
for filters in [32, 64, 128, 256]:
    x = layers.Conv2D(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)  # Added dense layer for projection
z_mean = layers.Dense(LATENT_DIM, activation='relu')(x)
z_log_var = layers.Dense(LATENT_DIM, activation='relu')(x)

# Define sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.shape(z_mean)[0], LATENT_DIM), mean=0., stddev=1.)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Define the decoder part of VAE
decoder_inputs = layers.Input(shape=(LATENT_DIM,))
x = layers.Dense(512, activation='relu')(decoder_inputs)  # Dense layer for decoding
x = layers.Dense(8*8*256, activation='relu')(x)  # Projection
x = layers.Reshape((8, 8, 256))(x)
for filters in [256, 128, 64, 32]:
    x = layers.Conv2DTranspose(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.UpSampling2D(size=(2, 2))(x)
decoder_outputs = layers.Conv2DTranspose(3, 4, activation='sigmoid', padding='same')(x)

# Define the VAE model
encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
decoder = models.Model(decoder_inputs, decoder_outputs, name='decoder')

outputs = decoder(encoder(encoder_inputs)[2])
vae = models.Model(encoder_inputs, outputs, name='vae')

# Define the loss function for VAE
def vae_loss(inputs, outputs, z_mean, z_log_var):
    reconstruction_loss = tf.keras.losses.binary_crossentropy(tf.keras.backend.flatten(inputs), tf.keras.backend.flatten(outputs))
    reconstruction_loss *= IMAGE_SIZE[0] * IMAGE_SIZE[1] * 3
    kl_loss = 1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var)
    kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = tf.keras.backend.mean(reconstruction_loss + RECONSTRUCTION_LOSS_MULTIPLIER * kl_loss)
    return vae_loss

# Compile the VAE model
vae.compile(optimizer='adam', loss=lambda x, y: vae_loss(encoder_inputs, outputs, z_mean, z_log_var))

# Train the VAE model
vae.fit(train_images, train_images, epochs=EPOCHS, batch_size=BATCH_SIZE)

# Extract embeddings from the encoder part
embeddings = encoder.predict(train_images)[0]  # Assuming you want z_mean

# Save or use embeddings as needed
np.save("PD.npy", embeddings)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Load the embeddings from the .npy file
embeddings = np.load("PD.npy")

# Assuming the embeddings are 3-dimensional
x = embeddings[:, 0]
y = embeddings[:, 1]
z = embeddings[:, 2]

# Plot the embeddings in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, c='b', marker='o')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()

# AltPD_test (AltPD)

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define constants
IMAGE_SIZE = (128, 128)
LATENT_DIM = 32
BATCH_SIZE = 50
EPOCHS = 500
RECONSTRUCTION_LOSS_MULTIPLIER = 0.5

# Function to load and preprocess images
def load_and_preprocess_images(image_folder):
    image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=IMAGE_SIZE)
        img = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return np.array(images)

# Load and preprocess images
train_images = load_and_preprocess_images("AltPD_test_dsl")

# Define the encoder part of VAE
encoder_inputs = layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = encoder_inputs
for filters in [32, 64, 128, 256]:
    x = layers.Conv2D(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)  # Added dense layer for projection
z_mean = layers.Dense(LATENT_DIM, activation='relu')(x)
z_log_var = layers.Dense(LATENT_DIM, activation='relu')(x)

# Define sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.shape(z_mean)[0], LATENT_DIM), mean=0., stddev=1.)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Define the decoder part of VAE
decoder_inputs = layers.Input(shape=(LATENT_DIM,))
x = layers.Dense(512, activation='relu')(decoder_inputs)  # Dense layer for decoding
x = layers.Dense(8*8*256, activation='relu')(x)  # Projection
x = layers.Reshape((8, 8, 256))(x)
for filters in [256, 128, 64, 32]:
    x = layers.Conv2DTranspose(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.UpSampling2D(size=(2, 2))(x)
decoder_outputs = layers.Conv2DTranspose(3, 4, activation='sigmoid', padding='same')(x)

# Define the VAE model
encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
decoder = models.Model(decoder_inputs, decoder_outputs, name='decoder')

outputs = decoder(encoder(encoder_inputs)[2])
vae = models.Model(encoder_inputs, outputs, name='vae')

# Define the loss function for VAE
def vae_loss(inputs, outputs, z_mean, z_log_var):
    reconstruction_loss = tf.keras.losses.binary_crossentropy(tf.keras.backend.flatten(inputs), tf.keras.backend.flatten(outputs))
    reconstruction_loss *= IMAGE_SIZE[0] * IMAGE_SIZE[1] * 3
    kl_loss = 1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var)
    kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = tf.keras.backend.mean(reconstruction_loss + RECONSTRUCTION_LOSS_MULTIPLIER * kl_loss)
    return vae_loss

# Compile the VAE model
vae.compile(optimizer='adam', loss=lambda x, y: vae_loss(encoder_inputs, outputs, z_mean, z_log_var))

# Train the VAE model
vae.fit(train_images, train_images, epochs=EPOCHS, batch_size=BATCH_SIZE)

# Extract embeddings from the encoder part
embeddings = encoder.predict(train_images)[0]  # Assuming you want z_mean

# Save or use embeddings as needed
np.save("AltPD.npy", embeddings)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Load the embeddings from the .npy file
embeddings = np.load("AltPD.npy")

# Assuming the embeddings are 3-dimensional
x = embeddings[:, 0]
y = embeddings[:, 1]
z = embeddings[:, 2]

# Plot the embeddings in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, c='b', marker='o')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()

# outlier

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define constants
IMAGE_SIZE = (128, 128)
LATENT_DIM = 32
BATCH_SIZE = 50
EPOCHS = 500
RECONSTRUCTION_LOSS_MULTIPLIER = 0.5

# Function to load and preprocess images
def load_and_preprocess_images(image_folder):
    image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=IMAGE_SIZE)
        img = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return np.array(images)

# Load and preprocess images
train_images = load_and_preprocess_images("outlier_dsl")

# Define the encoder part of VAE
encoder_inputs = layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = encoder_inputs
for filters in [32, 64, 128, 256]:
    x = layers.Conv2D(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)  # Added dense layer for projection
z_mean = layers.Dense(LATENT_DIM, activation='relu')(x)
z_log_var = layers.Dense(LATENT_DIM, activation='relu')(x)

# Define sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.shape(z_mean)[0], LATENT_DIM), mean=0., stddev=1.)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Define the decoder part of VAE
decoder_inputs = layers.Input(shape=(LATENT_DIM,))
x = layers.Dense(512, activation='relu')(decoder_inputs)  # Dense layer for decoding
x = layers.Dense(8*8*256, activation='relu')(x)  # Projection
x = layers.Reshape((8, 8, 256))(x)
for filters in [256, 128, 64, 32]:
    x = layers.Conv2DTranspose(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.UpSampling2D(size=(2, 2))(x)
decoder_outputs = layers.Conv2DTranspose(3, 4, activation='sigmoid', padding='same')(x)

# Define the VAE model
encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
decoder = models.Model(decoder_inputs, decoder_outputs, name='decoder')

outputs = decoder(encoder(encoder_inputs)[2])
vae = models.Model(encoder_inputs, outputs, name='vae')

# Define the loss function for VAE
def vae_loss(inputs, outputs, z_mean, z_log_var):
    reconstruction_loss = tf.keras.losses.binary_crossentropy(tf.keras.backend.flatten(inputs), tf.keras.backend.flatten(outputs))
    reconstruction_loss *= IMAGE_SIZE[0] * IMAGE_SIZE[1] * 3
    kl_loss = 1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var)
    kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = tf.keras.backend.mean(reconstruction_loss + RECONSTRUCTION_LOSS_MULTIPLIER * kl_loss)
    return vae_loss

# Compile the VAE model
vae.compile(optimizer='adam', loss=lambda x, y: vae_loss(encoder_inputs, outputs, z_mean, z_log_var))

# Train the VAE model
vae.fit(train_images, train_images, epochs=EPOCHS, batch_size=BATCH_SIZE)

# Extract embeddings from the encoder part
embeddings = encoder.predict(train_images)[0]  # Assuming you want z_mean

# Save or use embeddings as needed
np.save("outlier.npy", embeddings)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Load the embeddings from the .npy file
embeddings = np.load("outlier.npy")

# Assuming the embeddings are 3-dimensional
x = embeddings[:, 0]
y = embeddings[:, 1]
z = embeddings[:, 2]

# Plot the embeddings in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, c='b', marker='o')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()

# uniform (Syn)

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define constants
IMAGE_SIZE = (128, 128)
LATENT_DIM = 32
BATCH_SIZE = 50
EPOCHS = 50
RECONSTRUCTION_LOSS_MULTIPLIER = 0.5

# Function to load and preprocess images
def load_and_preprocess_images(image_folder):
    image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=IMAGE_SIZE)
        img = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return np.array(images)

# Load and preprocess images
train_images = load_and_preprocess_images("uniform_dsl")

# Define the encoder part of VAE
encoder_inputs = layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = encoder_inputs
for filters in [32, 64, 128, 256]:
    x = layers.Conv2D(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)  # Added dense layer for projection
z_mean = layers.Dense(LATENT_DIM, activation='relu')(x)
z_log_var = layers.Dense(LATENT_DIM, activation='relu')(x)

# Define sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.shape(z_mean)[0], LATENT_DIM), mean=0., stddev=1.)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Define the decoder part of VAE
decoder_inputs = layers.Input(shape=(LATENT_DIM,))
x = layers.Dense(512, activation='relu')(decoder_inputs)  # Dense layer for decoding
x = layers.Dense(8*8*256, activation='relu')(x)  # Projection
x = layers.Reshape((8, 8, 256))(x)
for filters in [256, 128, 64, 32]:
    x = layers.Conv2DTranspose(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.UpSampling2D(size=(2, 2))(x)
decoder_outputs = layers.Conv2DTranspose(3, 4, activation='sigmoid', padding='same')(x)

# Define the VAE model
encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
decoder = models.Model(decoder_inputs, decoder_outputs, name='decoder')

outputs = decoder(encoder(encoder_inputs)[2])
vae = models.Model(encoder_inputs, outputs, name='vae')

# Define the loss function for VAE
def vae_loss(inputs, outputs, z_mean, z_log_var):
    reconstruction_loss = tf.keras.losses.binary_crossentropy(tf.keras.backend.flatten(inputs), tf.keras.backend.flatten(outputs))
    reconstruction_loss *= IMAGE_SIZE[0] * IMAGE_SIZE[1] * 3
    kl_loss = 1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var)
    kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = tf.keras.backend.mean(reconstruction_loss + RECONSTRUCTION_LOSS_MULTIPLIER * kl_loss)
    return vae_loss

# Compile the VAE model
vae.compile(optimizer='adam', loss=lambda x, y: vae_loss(encoder_inputs, outputs, z_mean, z_log_var))

# Train the VAE model
vae.fit(train_images, train_images, epochs=EPOCHS, batch_size=BATCH_SIZE)

# Extract embeddings from the encoder part
embeddings = encoder.predict(train_images)[0]  # Assuming you want z_mean

# Save or use embeddings as needed
np.save("uniform.npy", embeddings)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Load the embeddings from the .npy file
embeddings = np.load("uniform.npy")

# Assuming the embeddings are 3-dimensional
x = embeddings[:, 0]
y = embeddings[:, 1]
z = embeddings[:, 2]

# Plot the embeddings in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, c='b', marker='o')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()

# advpd

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define constants
IMAGE_SIZE = (128, 128)
LATENT_DIM = 32
BATCH_SIZE = 50
EPOCHS = 100
RECONSTRUCTION_LOSS_MULTIPLIER = 0.5

# Function to load and preprocess images
def load_and_preprocess_images(image_folder):
    image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=IMAGE_SIZE)
        img = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return np.array(images)

# Load and preprocess images
train_images = load_and_preprocess_images("advpd_dsl")

# Define the encoder part of VAE
encoder_inputs = layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = encoder_inputs
for filters in [32, 64, 128, 256]:
    x = layers.Conv2D(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)  # Added dense layer for projection
z_mean = layers.Dense(LATENT_DIM, activation='relu')(x)
z_log_var = layers.Dense(LATENT_DIM, activation='relu')(x)

# Define sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.shape(z_mean)[0], LATENT_DIM), mean=0., stddev=1.)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Define the decoder part of VAE
decoder_inputs = layers.Input(shape=(LATENT_DIM,))
x = layers.Dense(512, activation='relu')(decoder_inputs)  # Dense layer for decoding
x = layers.Dense(8*8*256, activation='relu')(x)  # Projection
x = layers.Reshape((8, 8, 256))(x)
for filters in [256, 128, 64, 32]:
    x = layers.Conv2DTranspose(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.UpSampling2D(size=(2, 2))(x)
decoder_outputs = layers.Conv2DTranspose(3, 4, activation='sigmoid', padding='same')(x)

# Define the VAE model
encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
decoder = models.Model(decoder_inputs, decoder_outputs, name='decoder')

outputs = decoder(encoder(encoder_inputs)[2])
vae = models.Model(encoder_inputs, outputs, name='vae')

# Define the loss function for VAE
def vae_loss(inputs, outputs, z_mean, z_log_var):
    reconstruction_loss = tf.keras.losses.binary_crossentropy(tf.keras.backend.flatten(inputs), tf.keras.backend.flatten(outputs))
    reconstruction_loss *= IMAGE_SIZE[0] * IMAGE_SIZE[1] * 3
    kl_loss = 1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var)
    kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = tf.keras.backend.mean(reconstruction_loss + RECONSTRUCTION_LOSS_MULTIPLIER * kl_loss)
    return vae_loss

# Compile the VAE model
vae.compile(optimizer='adam', loss=lambda x, y: vae_loss(encoder_inputs, outputs, z_mean, z_log_var))

# Train the VAE model
vae.fit(train_images, train_images, epochs=EPOCHS, batch_size=BATCH_SIZE)

# Extract embeddings from the encoder part
embeddings = encoder.predict(train_images)[0]  # Assuming you want z_mean

# Save or use embeddings as needed
np.save("advpd.npy", embeddings)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Load the embeddings from the .npy file
embeddings = np.load("advpd.npy")

# Assuming the embeddings are 3-dimensional
x = embeddings[:, 0]
y = embeddings[:, 1]
z = embeddings[:, 2]

# Plot the embeddings in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, c='b', marker='o')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()

# img_noise_train (NPD)

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define constants
IMAGE_SIZE = (128, 128)
LATENT_DIM = 32
BATCH_SIZE = 50
EPOCHS = 1000
RECONSTRUCTION_LOSS_MULTIPLIER = 0.5

# Function to load and preprocess images
def load_and_preprocess_images(image_folder):
    image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder)]
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=IMAGE_SIZE)
        img = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return np.array(images)

# Load and preprocess images
train_images = load_and_preprocess_images("img_noise_train_dsl")

# Define the encoder part of VAE
encoder_inputs = layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
x = encoder_inputs
for filters in [32, 64, 128, 256]:
    x = layers.Conv2D(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)  # Added dense layer for projection
z_mean = layers.Dense(LATENT_DIM, activation='relu')(x)
z_log_var = layers.Dense(LATENT_DIM, activation='relu')(x)

# Define sampling layer
def sampling(args):
    z_mean, z_log_var = args
    epsilon = tf.keras.backend.random_normal(shape=(tf.shape(z_mean)[0], LATENT_DIM), mean=0., stddev=1.)
    return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_var])

# Define the decoder part of VAE
decoder_inputs = layers.Input(shape=(LATENT_DIM,))
x = layers.Dense(512, activation='relu')(decoder_inputs)  # Dense layer for decoding
x = layers.Dense(8*8*256, activation='relu')(x)  # Projection
x = layers.Reshape((8, 8, 256))(x)
for filters in [256, 128, 64, 32]:
    x = layers.Conv2DTranspose(filters, 4, activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.UpSampling2D(size=(2, 2))(x)
decoder_outputs = layers.Conv2DTranspose(3, 4, activation='sigmoid', padding='same')(x)

# Define the VAE model
encoder = models.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
decoder = models.Model(decoder_inputs, decoder_outputs, name='decoder')

outputs = decoder(encoder(encoder_inputs)[2])
vae = models.Model(encoder_inputs, outputs, name='vae')

# Define the loss function for VAE
def vae_loss(inputs, outputs, z_mean, z_log_var):
    reconstruction_loss = tf.keras.losses.binary_crossentropy(tf.keras.backend.flatten(inputs), tf.keras.backend.flatten(outputs))
    reconstruction_loss *= IMAGE_SIZE[0] * IMAGE_SIZE[1] * 3
    kl_loss = 1 + z_log_var - tf.keras.backend.square(z_mean) - tf.keras.backend.exp(z_log_var)
    kl_loss = tf.keras.backend.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = tf.keras.backend.mean(reconstruction_loss + RECONSTRUCTION_LOSS_MULTIPLIER * kl_loss)
    return vae_loss

# Compile the VAE model
vae.compile(optimizer='adam', loss=lambda x, y: vae_loss(encoder_inputs, outputs, z_mean, z_log_var))

# Train the VAE model
vae.fit(train_images, train_images, epochs=EPOCHS, batch_size=BATCH_SIZE)

# Extract embeddings from the encoder part
embeddings = encoder.predict(train_images)[0]  # Assuming you want z_mean

# Save or use embeddings as needed
np.save("img.npy", embeddings)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Load the embeddings from the .npy file
embeddings = np.load("img.npy")

# Assuming the embeddings are 3-dimensional
x = embeddings[:, 0]
y = embeddings[:, 1]
z = embeddings[:, 2]

# Plot the embeddings in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, c='b', marker='o')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()

# Plotting

In [ ]:
import numpy as np
from multiprocessing import Pool

# Load datasets from the files
true_train_zs = np.load("train.npy")
true_val_zs = np.load("val.npy")
pd_zs = np.load("PD.npy")

def base_kernel(x, y, sigma):
    norm_square = np.linalg.norm(x - y) ** 2
    sigma_square = sigma ** 2
    return np.exp(-norm_square / (2 * sigma_square))

def composite_kernel(x, y, sigmas):
    result = 0
    for sigma in sigmas:
        result += base_kernel(x, y, sigma)
    return result

def compute_mmd(dataset_x, dataset_y, sigmas=[1, 5, 10, 15, 20]):
    result = 0
    n = len(dataset_x)
    m = len(dataset_y)
    
    for i in range(n):
        for j in range(n):
            result += 1. / (n ** 2) * composite_kernel(dataset_x[i], dataset_x[j], sigmas)
    
    for i in range(n):
        for j in range(m):
            result -= 2. / (n * m) * composite_kernel(dataset_x[i], dataset_y[j], sigmas)
    
    for i in range(m):
        for j in range(m):
            result += 1. / (m ** 2) * composite_kernel(dataset_y[i], dataset_y[j], sigmas)
    
    return np.sqrt(result)

draws = 20
samples_per_draw = 100

for _ in range(draws):
    train_subs = true_train_zs[np.random.choice(true_train_zs.shape[0], samples_per_draw, replace=False)]

def test_mmd(train, val, test, draws=20):
    res = compute_mmd(train_subs, test)
    return res

def test_mmd_wrapper(i):
    return test_mmd(true_train_zs, true_val_zs, pd_zs[i+100:i+200])

p = Pool(20)
test_mmds = p.map(test_mmd_wrapper, [i for i in range(250)])
p.close()
p.join()

print(test_mmds)


In [ ]:
plt.plot(test_mmds)
plt.legend(['PD'])
plt.yticks(np.arange(0.0, 3.1, 0.5))
plt.show()

In [ ]:
import numpy as np
from multiprocessing import Pool

# Load datasets from the files
true_train_zs = np.load("train.npy")
true_val_zs = np.load("val.npy")
pd_zs = np.load("AltPD.npy")

def base_kernel(x, y, sigma):
    norm_square = np.linalg.norm(x - y) ** 2
    sigma_square = sigma ** 2
    return np.exp(-norm_square / (2 * sigma_square))

def composite_kernel(x, y, sigmas):
    result = 0
    for sigma in sigmas:
        result += base_kernel(x, y, sigma)
    return result

def compute_mmd(dataset_x, dataset_y, sigmas=[1, 5, 10, 15, 20]):
    result = 0
    n = len(dataset_x)
    m = len(dataset_y)
    
    for i in range(n):
        for j in range(n):
            result += 1. / (n ** 2) * composite_kernel(dataset_x[i], dataset_x[j], sigmas)
    
    for i in range(n):
        for j in range(m):
            result -= 2. / (n * m) * composite_kernel(dataset_x[i], dataset_y[j], sigmas)
    
    for i in range(m):
        for j in range(m):
            result += 1. / (m ** 2) * composite_kernel(dataset_y[i], dataset_y[j], sigmas)
    
    return np.sqrt(result)

draws = 20
samples_per_draw = 100

for _ in range(draws):
    train_subs = true_train_zs[np.random.choice(true_train_zs.shape[0], samples_per_draw, replace=False)]

def test_mmd(train, val, test, draws=20):
    res = compute_mmd(train_subs, test)
    return res

def test_mmd_wrapper(i):
    return test_mmd(true_train_zs, true_val_zs, pd_zs[i+100:i+200])

p = Pool(20)
altpd_test_mmds = p.map(test_mmd_wrapper, [i for i in range(250)])
p.close()
p.join()

print(altpd_test_mmds)


In [ ]:
plt.plot(altpd_test_mmds)
plt.legend(['AltPD'])
plt.yticks(np.arange(0.0, 3.1, 0.5))
plt.show()

In [ ]:
import numpy as np
from multiprocessing import Pool

# Load datasets from the files
true_train_zs = np.load("train.npy")
true_val_zs = np.load("val.npy")
uniform_zs = np.load("uniform.npy")

def base_kernel(x, y, sigma):
    norm_square = np.linalg.norm(x - y) ** 2
    sigma_square = sigma ** 2
    return np.exp(-norm_square / (2 * sigma_square))

def composite_kernel(x, y, sigmas):
    result = 0
    for sigma in sigmas:
        result += base_kernel(x, y, sigma)
    return result

def compute_mmd(dataset_x, dataset_y, sigmas=[1, 5, 10, 15, 20]):
    result = 0
    n = len(dataset_x)
    m = len(dataset_y)
    
    for i in range(n):
        for j in range(n):
            result += 1. / (n ** 2) * composite_kernel(dataset_x[i], dataset_x[j], sigmas)
    
    for i in range(n):
        for j in range(m):
            result -= 2. / (n * m) * composite_kernel(dataset_x[i], dataset_y[j], sigmas)
    
    for i in range(m):
        for j in range(m):
            result += 1. / (m ** 2) * composite_kernel(dataset_y[i], dataset_y[j], sigmas)
    
    return np.sqrt(result)

draws = 20
samples_per_draw = 100

for _ in range(draws):
    train_subs = true_train_zs[np.random.choice(true_train_zs.shape[0], samples_per_draw, replace=False)]

def test_mmd(train, val, test, draws=20):
    res = compute_mmd(train_subs, test)
    return res

def unif_mmd_wrapper(i):
    return test_mmd(true_train_zs, true_val_zs, uniform_zs[i+100:i+200])

p = Pool(20)
unif_mmds = p.map(unif_mmd_wrapper, [i for i in range(250)])
p.close()
p.join()

print(unif_mmds)


In [ ]:
plt.plot(unif_mmds)
plt.legend(['Syn'])
plt.yticks(np.arange(0.0, 3.1, 0.5))
plt.show()

In [ ]:
import numpy as np
from multiprocessing import Pool

# Load datasets from the files
true_train_zs = np.load("train.npy")
true_val_zs = np.load("val.npy")
advpd_zs = np.load("advpd.npy")

def base_kernel(x, y, sigma):
    norm_square = np.linalg.norm(x - y) ** 2
    sigma_square = sigma ** 2
    return np.exp(-norm_square / (2 * sigma_square))

def composite_kernel(x, y, sigmas):
    result = 0
    for sigma in sigmas:
        result += base_kernel(x, y, sigma)
    return result

def compute_mmd(dataset_x, dataset_y, sigmas=[1, 5, 10, 15, 20]):
    result = 0
    n = len(dataset_x)
    m = len(dataset_y)
    
    for i in range(n):
        for j in range(n):
            result += 1. / (n ** 2) * composite_kernel(dataset_x[i], dataset_x[j], sigmas)
    
    for i in range(n):
        for j in range(m):
            result -= 2. / (n * m) * composite_kernel(dataset_x[i], dataset_y[j], sigmas)
    
    for i in range(m):
        for j in range(m):
            result += 1. / (m ** 2) * composite_kernel(dataset_y[i], dataset_y[j], sigmas)
    
    return np.sqrt(result)

draws = 20
samples_per_draw = 100

for _ in range(draws):
    train_subs = true_train_zs[np.random.choice(true_train_zs.shape[0], samples_per_draw, replace=False)]

def test_mmd(train, val, test, draws=20):
    res = compute_mmd(train_subs, test)
    return res

def pap_mmd_wrapper(i):
    return test_mmd(true_train_zs, true_val_zs, advpd_zs[i+100:i+200])

p = Pool(20)
pap_mmds = p.map(pap_mmd_wrapper, [i for i in range(250)])
p.close()
p.join()

print(pap_mmds)


In [ ]:
plt.plot(pap_mmds)
plt.legend(['AdvPD'])
plt.yticks(np.arange(0.0, 3.1, 0.5))
plt.show()

In [ ]:
import numpy as np
from multiprocessing import Pool

# Load datasets from the files
true_train_zs = np.load("train.npy")
true_val_zs = np.load("val.npy")
img_noise_zs = np.load("img.npy")

def base_kernel(x, y, sigma):
    norm_square = np.linalg.norm(x - y) ** 2
    sigma_square = sigma ** 2
    return np.exp(-norm_square / (2 * sigma_square))

def composite_kernel(x, y, sigmas):
    result = 0
    for sigma in sigmas:
        result += base_kernel(x, y, sigma)
    return result

def compute_mmd(dataset_x, dataset_y, sigmas=[1, 5, 10, 15, 20]):
    result = 0
    n = len(dataset_x)
    m = len(dataset_y)
    
    for i in range(n):
        for j in range(n):
            result += 1. / (n ** 2) * composite_kernel(dataset_x[i], dataset_x[j], sigmas)
    
    for i in range(n):
        for j in range(m):
            result -= 2. / (n * m) * composite_kernel(dataset_x[i], dataset_y[j], sigmas)
    
    for i in range(m):
        for j in range(m):
            result += 1. / (m ** 2) * composite_kernel(dataset_y[i], dataset_y[j], sigmas)
    
    return np.sqrt(result)

draws = 20
samples_per_draw = 100

for _ in range(draws):
    train_subs = true_train_zs[np.random.choice(true_train_zs.shape[0], samples_per_draw, replace=False)]

def test_mmd(train, val, test, draws=20):
    res = compute_mmd(train_subs, test)
    return res

def img_mmd_wrapper(i):
    return test_mmd(true_train_zs, true_val_zs, img_noise_zs[i+100:i+200])

p = Pool(20)
img_mmds = p.map(img_mmd_wrapper, [i for i in range(250)])
p.close()
p.join()

print(img_mmds)


In [ ]:
plt.plot(img_mmds)
plt.legend(['NPD'])
plt.yticks(np.arange(0.0, 3.1, 0.5))
plt.show()

In [ ]:
plt.plot(test_mmds)
plt.plot(unif_mmds)
plt.plot(img_mmds)
plt.plot(pap_mmds)

plt.legend(['PD', 'Syn', 'NPD', 'AdvPD'])
plt.yticks(np.arange(0.0, 3.1, 0.5))
plt.show()

In [ ]:
plt.plot(test_mmds)
plt.plot(unif_mmds)
plt.plot(img_mmds)
plt.plot(pap_mmds)
plt.plot(altpd_test_mmds)

plt.legend(['PD', 'Syn', 'NPD', 'AdvPD', 'AltPD'])
plt.yticks(np.arange(0.0, 3.1, 0.5))
plt.show()